In [1]:
import pandas as pd
import numpy as np
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split
import os

In [2]:
os.chdir('data')
log = pd.read_csv('log.tsv',delimiter="\t")
events = pd.read_csv('events.tsv', delimiter="\t")
users = pd.read_csv('users.tsv', delimiter="\t")
os.chdir('..')

In [3]:
print(log.shape)
log.head()

(1452089, 7)


,user_id,event_id,time_stamp,action_type,num_of_people,payment_method,total_price
0,1,6261,2015-03-19 09:15:50,3,1.0,クレカ,4000.0
1,1,127600,2017-06-05 12:33:17,1,NaN,NaN,NaN
2,1,127600,2017-06-05 12:37:09,1,NaN,NaN,NaN
3,1,125296,2017-06-06 14:01:27,1,NaN,NaN,NaN
4,1,125128,2017-06-06 14:03:57,1,NaN,NaN,NaN


In [4]:
print(events.shape)
events.head()

(158392, 11)


,event_id,female_age_lower,female_age_upper,male_age_lower,male_age_upper,event_start_at,prefecture,first_published_at,female_price,male_price,interest
0,1,20,99.0,25,44.0,2015-03-08 18:00:00,宮城県,NaN,1500.0,5000.0,NaN
1,2,30,99.0,30,49.0,2015-03-01 15:00:00,富山県,NaN,1000.0,6500.0,NaN
2,3,22,99.0,24,39.0,2015-03-07 19:00:00,富山県,NaN,1000.0,6000.0,NaN
3,4,20,99.0,25,44.0,2015-03-08 18:00:00,新潟県,NaN,1000.0,6000.0,NaN
4,5,20,99.0,20,39.0,2015-03-14 19:00:00,新潟県,NaN,1000.0,6000.0,NaN


In [5]:
print(users.shape)
users.head()

(69667, 5)


,user_id,age,gender,prefecture,created_on
0,1,34,女性,静岡県,2012-06-26
1,2,31,男性,鳥取県,2012-07-12
2,3,32,男性,東京都,2012-07-15
3,4,30,男性,東京都,2012-07-17
4,5,29,女性,埼玉県,2012-07-17


http://surprise.readthedocs.io/en/stable/getting_started.html  
# Tutorialどおりにやってみる


In [6]:
import pandas as pd

from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate

In [7]:
table = log.loc[:,["user_id","event_id","action_type"]]
table.head()

,user_id,event_id,action_type
0,1,6261,3
1,1,127600,1
2,1,127600,1
3,1,125296,1
4,1,125128,1


In [8]:
reader = Reader(rating_scale=(1,3))
data = Dataset.load_from_df(table, reader)

In [9]:
train = data.build_full_trainset()

## チューニング

In [16]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import GridSearchCV



param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3, refit=True)

gs.fit(data)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

0.6107718905574119
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}


In [10]:
algo = SVD(n_epochs = 10, lr_all = 0.005, reg_all = 0.4)


In [11]:
algo.fit(train)

## 予測

In [12]:
os.chdir('data')
test = pd.read_csv('test.tsv',delimiter="\t")
os.chdir('..')

In [13]:
print(test.shape)
test.head()

(2886, 1)


,user_id
0,1
1,6
2,23
3,43
4,46


In [16]:
#tutorialに載ってる予測のしかた
uid = str(1)
iid = str(1)

pred = algo.predict(uid, iid, verbose=True)

#test全体に対して予測するのどうやるんだ・・・？
#for文？

user: 1          item: 1          r_ui = None   est = 1.59   {'was_impossible': False}
